In [6]:
# ---------------------------
# Lenie Silverman
# CS-340
# Grazioso Salvare Dashboard (Project Two)
# 
# ---------------------------

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

import pandas as pd
import base64
import plotly.express as px
import glob
import os

from AnimalShelter import AnimalShelter

JupyterDash.infer_jupyter_proxy_config()

# ---------------------------
# MongoDB Connection
# ---------------------------
username = "aacuser"
password = "ChangeMe123"
host = "127.0.0.1"
database = "aac"
collection = "animals"

# NOTE: your AnimalShelter __init__ likely takes username/password/host/port/db/collection
# Use the signature you built in Project One. This is the common pattern:
shelter = AnimalShelter(username, password, port=27017)

# ---------------------------
# Load Initial Data
# ---------------------------
try:
    df = pd.DataFrame.from_records(shelter.read({}))
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
except Exception as e:
    print("Error loading data:", e)
    df = pd.DataFrame()

# ---------------------------
# Helper: DataTable Columns
# ---------------------------
def make_columns(dataframe: pd.DataFrame):
    return [{"name": col, "id": col, "deletable": False, "selectable": True} for col in dataframe.columns]

# ---------------------------
# Logo (auto-find)
# ---------------------------
encoded_image = None
matches = glob.glob("**/*logo*.png", recursive=True) + glob.glob("**/*Logo*.png", recursive=True)

if matches:
    image_filename = matches[0]
    with open(image_filename, "rb") as f:
        encoded_image = base64.b64encode(f.read()).decode("utf-8")
else:
    print("Logo file not found. (This is OK; dashboard will show text instead.)")

# ---------------------------
# App
# ---------------------------
app = JupyterDash(__name__)

# ---------------------------
# Layout
# ---------------------------
header_left = (
    html.Img(src=f"data:image/png;base64,{encoded_image}", height=80)
    if encoded_image
    else html.Div("Grazioso Salvare", style={"fontWeight": "bold", "fontSize": "24px"})
)

app.layout = html.Div([
    # Header: logo + name identifier
    html.Div(
        style={"display": "flex", "alignItems": "center", "gap": "20px"},
        children=[
            header_left,
            html.Div([
                html.H2("Grazioso Salvare Dashboard", style={"margin": "0"}),
                html.Div("Lenie Silverman - SNHU CS-340", style={"opacity": 0.8}),
            ])
        ],
    ),

    html.Hr(),

    # Filter controls
    dcc.RadioItems(
        id="rescue-type-filter",
        options=[
            {"label": "Reset (All)", "value": "Reset"},
            {"label": "Water Rescue", "value": "Water"},
            {"label": "Mountain / Wilderness Rescue", "value": "Mountain"},
            {"label": "Disaster / Individual Tracking", "value": "Disaster"},
        ],
        value="Reset",
        labelStyle={"display": "inline-block", "marginRight": "18px"},
    ),

    html.Hr(),

    # Data table
    dash_table.DataTable(
        id="datatable-id",
        columns=make_columns(df),
        data=df.to_dict("records"),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_cell={"whiteSpace": "normal", "height": "auto", "minWidth": "110px", "width": "110px", "maxWidth": "110px"},
    ),

    html.Hr(),

    # Chart + Map together
    html.Div(
        style={"display": "flex", "gap": "20px"},
        children=[
            html.Div(style={"flex": "1"}, children=[dcc.Graph(id="graph-id")]),
            html.Div(style={"flex": "1"}, children=[html.Div(id="map-id")]),
        ],
    ),
])

# ---------------------------
# Controller Callbacks
# ---------------------------

@app.callback(
    Output("datatable-id", "data"),
    Output("datatable-id", "columns"),
    Input("rescue-type-filter", "value"),
)
def update_table(filter_type):
    """Pull filtered data FROM MongoDB and update the table."""
    if filter_type == "Water":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "Mountain":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    elif filter_type == "Disaster":
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    else:
        query = {}

    dff = pd.DataFrame.from_records(shelter.read(query))
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    # if empty, keep columns stable to avoid table crash
    if dff.empty:
        dff = pd.DataFrame(columns=df.columns)

    return dff.to_dict("records"), make_columns(dff)


@app.callback(
    Output("graph-id", "figure"),
    Input("datatable-id", "derived_virtual_data"),
)
def update_graph(viewData):
    """Update second chart from the currently visible table data."""
    if not viewData:
        # blank fig
        return px.histogram(pd.DataFrame({"breed": []}), x="breed", title="Breed Distribution")

    dff = pd.DataFrame(viewData)
    if "breed" in dff.columns:
        return px.histogram(dff, x="breed", title="Breed Distribution")
    else:
        # fallback if breed column is missing
        return px.histogram(dff, title="Distribution")


@app.callback(
    Output("map-id", "children"),
    Input("datatable-id", "derived_virtual_data"),
    Input("datatable-id", "derived_virtual_selected_rows"),
)
def update_map(viewData, selected_rows):
    """
    Update geolocation map from selected row.
    Expects columns: location_lat and location_long (common in AAC dataset).
    """
    default_center = [30.75, -97.48]

    # Base map if no data
    if not viewData:
        return dl.Map(
            style={"width": "100%", "height": "500px"},
            center=default_center,
            zoom=10,
            children=[dl.TileLayer()],
        )

    dff = pd.DataFrame(viewData)
    if dff.empty:
        return dl.Map(
            style={"width": "100%", "height": "500px"},
            center=default_center,
            zoom=10,
            children=[dl.TileLayer()],
        )

    row = selected_rows[0] if selected_rows else 0
    if row >= len(dff):
        row = 0

    # Use safe getters
    lat = dff.iloc[row].get("location_lat", default_center[0])
    lon = dff.iloc[row].get("location_long", default_center[1])

    # Some datasets store coordinates as strings—convert if needed
    try:
        lat = float(lat)
        lon = float(lon)
    except Exception:
        lat, lon = default_center

    name = dff.iloc[row].get("name", "No Name")
    breed = dff.iloc[row].get("breed", "Unknown Breed")

    return dl.Map(
        style={"width": "100%", "height": "500px"},
        center=[lat, lon],
        zoom=10,
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(breed),
                    dl.Popup([
                        html.H4("Selected Animal"),
                        html.P(f"Name: {name}"),
                        html.P(f"Breed: {breed}"),
                        html.P(f"Lat: {lat}, Long: {lon}"),
                    ]),
                ],
            ),
        ],
    )

# ---------------------------
# Run Server
# ---------------------------
if __name__ == "__main__":
    app.run_server(mode="jupyterlab", port=8054, debug=True)


MongoDB connected successfully!
